迭代器和生成器
- 可迭代对象和迭代器

容器：<br>
Python中一切皆对象，对象的抽象是类，对象的集合是容器。
- 列表、元组、字典、集合都是容器
- 容器都是可以迭代的
- 迭代器（iterator）提供一个next方法。
- 可迭代对象可以通过iter()函数返回一个迭代器，再通过next()函数就可以实现遍历。
- for in 语句将这个过程隐式化。

In [2]:
from collections.abc import Iterable


# 判断一个对象是否可迭代
def is_iterable(param):
    try:
        iter(param)
        return True
    except TypeError:
        return False
# 或者


def isNot_iterable(param):
    return isinstance(param, Iterable)


parms = [1234, '1234', [1, 2, 3, 4], set(
    [1, 2, 3, 4]), {1: 1, 2: 2, 3: 3, 4: 4}, (1, 2, 3, 4)]

for param in parms:
    print('{} is iterable? {} {}'.format(
        param, is_iterable(param), isNot_iterable(param)))


1234 is iterable? False False
1234 is iterable? True True
[1, 2, 3, 4] is iterable? True True
{1, 2, 3, 4} is iterable? True True
{1: 1, 2: 2, 3: 3, 4: 4} is iterable? True True
(1, 2, 3, 4) is iterable? True True


-<br>
- 生成器<br>
懒人版本的迭代器


In [4]:
# 在迭代器中，如果我们想要枚举它的元素，这些元素要事先生成。举例如下：
import os
import psutil

# 显示当前python程序占用的内存大小
def show_memory_info(hint):
    pid = os.getpid()
    p=psutil.Process(pid)

    info = p.memory_full_info()
    memory=info.uss/1024./1024
    print('{} memory used: {} MB'.format(hint,memory))

def test_iterator():
    show_memory_info('initing iterator')
    list_1=[i for i in range(100000000)]
    show_memory_info('after iterator initiated')
    print(sum(list_1))
    show_memory_info('after sum called')


def test_generator():
    show_memory_info('initing generator')
    list_2=(i for i in range(100000000))
    show_memory_info('after generator initiated')
    print(sum(list_2))
    show_memory_info('after sum called')

%time test_iterator()
%time test_generator()

initing iterator memory used: 13.78515625 MB
after iterator initiated memory used: 3885.16015625 MB
4999999950000000
after sum called memory used: 3885.16015625 MB
CPU times: total: 11.5 s
Wall time: 11.9 s
initing generator memory used: 14.23046875 MB
after generator initiated memory used: 14.23046875 MB
4999999950000000
after sum called memory used: 14.23828125 MB
CPU times: total: 8.44 s
Wall time: 8.61 s


迭代器，生成的元素都会保存到内存中，占用巨量的内存，内存不够可能会出现OOM错误。<br>
有时候，我们并不需要在内存中保存这么多东西，比如对元素和，我们只知道每个元素在相加的那一刻是多少就行了，用完就可以扔掉了。<br>
于是，生成器的概念应运而生，在调用next()函数的时候，才会生成下一个变量。


生成器，还能玩出什么花样？